In [98]:
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm
from shapely.geometry import box, Polygon
from shapely.ops import transform
from shapely import wkt
import matplotlib.pyplot as plt
import os, sys
import pyproj

#Enables imports from src directory in notebooks
sys.path.insert(0, os.path.abspath('../src'))
sys.path.insert(0, os.path.abspath('../src/data_handling'))
#Auto update imports when python files in src is updated
%load_ext autoreload
%autoreload 2

from satellite_images import read_sat_images_file
from utils import boundingBox, write_polygons_to_shp, plot_polygons, png_to_geotiff


data_location = "E:/Universitetet i Agder/Mikkel Andreas Kvande - kornmo-data-files/raw-data"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
def get_farmer_centroid(nrows=None):
    farmer_centroid = pd.read_csv(os.path.join(data_location, 'farm-information/centroid_coordinates_new.csv'), delimiter=',', nrows=nrows)
    columns_to_keep = ['orgnr', 'latitude', 'longitude', 'kommunenr']
    farmer_centroid = farmer_centroid.filter(columns_to_keep)
    farmer_centroid = farmer_centroid.dropna()
    farmer_centroid['orgnr'] = farmer_centroid['orgnr'].astype(str)
    farmer_centroid['latitude'] = farmer_centroid['latitude'].astype(float)
    farmer_centroid['longitude'] = farmer_centroid['longitude'].astype(float)
    farmer_centroid['kommunenr'] = farmer_centroid['kommunenr'].astype(int)
    return farmer_centroid

def get_polygon_data(nrows=None):
    
    soilquality = pd.read_csv(os.path.join(data_location, 'soil-data/jordsmonn_geometry.csv'), dtype=str, nrows=nrows)
    soilquality = soilquality.dropna()
    soilquality['geometry'] = soilquality['geometry'].apply(wkt.loads)
    geo_soilquality = gpd.GeoDataFrame(soilquality, crs='epsg:4326')
    geo_soilquality['KOMID'] = geo_soilquality['KOMID'].astype(int)
    
    return geo_soilquality

def get_disp_eiendommer():
    disp_eien = gpd.read_file(os.path.join(data_location, 'farm-information/farm-properties/disposed-properties-previous-students/disponerte_eiendommer.gpkg', layer='disponerte_eiendommer'))
    disp_eien = disp_eien.dropna()
    disp_eien['orgnr'] = disp_eien['orgnr'].astype(str)

    return disp_eien

def get_combined_satellite_data():
    sat_images0 = read_sat_images_file('sentinel_100x100_0.h5')
    sat_images1 = read_sat_images_file('sentinel_100x100_1.h5')
    in_first = set(sat_images0)
    in_second = set(sat_images1)

    in_second_but_not_in_first = in_second - in_first

    result = list(sat_images0) + list(in_second_but_not_in_first)

    return result


def get_orgnrs_by_municipal(municipal_nr, orgnr_dataframe):
    return orgnr_dataframe.loc[orgnr_dataframe['kommunenr'] == municipal_nr]


def get_polygons_by_municipal(polygons, municipal_nr):
    polygons_by_muni = polygons.loc[polygons['KOMID'] == municipal_nr]
    polygons_list = list(polygons_by_muni['geometry'])
    return polygons_list, polygons_by_muni.index.values.tolist()

def convert_crs(polygons):
    project = pyproj.Transformer.from_proj(pyproj.Proj('epsg:25833'), pyproj.Proj('epsg:4326'), always_xy=True)
    return [transform(project.transform, poly) for poly in polygons]



In [100]:
print("Retrieving farmer centroid data, satellite data and polygon data")
farmer_centroid = get_farmer_centroid()
polygons = get_polygon_data()

sat_orgnr = np.array(get_combined_satellite_data())
farm_orgnr = np.array(list(farmer_centroid['orgnr']))
intersection = np.intersect1d(sat_orgnr, farm_orgnr)

filtered_farmer_centroid = farmer_centroid[farmer_centroid['orgnr'].isin(intersection)]
filtered_satellite_data = intersection[:]

print(f"Farmer elevation shape: {filtered_farmer_centroid.shape}")
print(f"Length of satellite data after filtering: {len(filtered_satellite_data)}")

municipal_nrs_farm = np.array(list(set(filtered_farmer_centroid['kommunenr'])))
municipal_nrs_sat = np.array(list(set(polygons['KOMID'])))
intersection = np.intersect1d(municipal_nrs_farm, municipal_nrs_sat)

filtered_farmer_centroid = filtered_farmer_centroid[filtered_farmer_centroid['kommunenr'].isin(intersection)]
filtered_polygons = polygons[polygons['KOMID'].isin(intersection)]

print(f"Farmer centroid shape after second filtering: {filtered_farmer_centroid.shape}")
print(f"Polygon data shape after second filtering: {filtered_polygons.shape}")



Retrieving farmer centroid data, satellite data and polygon data


100%|██████████| 3477/3477 [00:00<00:00, 36460.35it/s]


Farmer elevation shape: (6738, 4)
Length of satellite data after filtering: 6708
Farmer elevation shape after second filtering: (6738, 4)
Polygon data shape after second filtering: (585241, 3)


In [102]:

municipal_nrs = list(set(filtered_farmer_centroid['kommunenr']))
intersections = {}
intersections_df = []
for municipal_nr in tqdm(municipal_nrs):
    orgnrs_by_municipal_nr = get_orgnrs_by_municipal(municipal_nr, filtered_farmer_centroid)
    org_nrs_list = list(orgnrs_by_municipal_nr['orgnr'])

    polygons_by_municipal_nr, polygon_indices = get_polygons_by_municipal(filtered_polygons, municipal_nr)

    transformed_polygons = convert_crs(polygons_by_municipal_nr)
    #write_polygons_to_shp(transformed_polygons, str(municipal_nr))
    #polygons_plt = [Polygon(mapping(poly)['coordinates'][0][0]) for poly in transformed_polygons]
    #plot_polygons(polygons_plt)
    org_dict = {}
    for orgnr in org_nrs_list:
        row = orgnrs_by_municipal_nr.loc[orgnrs_by_municipal_nr['orgnr'] == orgnr].iloc[0]

        bounding_box = boundingBox(row['latitude'], row['longitude'], 1)
        bbox = box(bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3])
        
        for i, polygon in enumerate(transformed_polygons):
            if polygon.intersects(bbox):
                new_row = [municipal_nr, orgnr, polygon_indices[i]]
                intersections_df.append(new_row)
                if orgnr in org_dict:
                    org_dict[orgnr] += 1
                else:
                    org_dict[orgnr] = 0
    intersections[municipal_nr] = org_dict


intersections_df = pd.DataFrame(intersections_df, columns=['municipal_nr', 'orgnr', 'field_id'])
intersections_df.to_csv(os.path.join(data_location, 'farm-information/fields_per_farm.csv'))
print(intersections_df.head())
print(intersections_df.shape)




    

100%|██████████| 144/144 [02:47<00:00,  1.17s/it]


   municipal_nr      orgnr  field_id
0          1579  969990873    581984
1          1579  969990873    581992
2          1579  969990873    581993
3          1579  969990873    581994
4          1579  969990873    581996
(1109422, 3)


In [33]:
print(len(intersections))
municipal_nr = 101
orgnr = '918312145'
orgnrs_by_municipal_nr = get_orgnrs_by_municipal(municipal_nr, farmer_centroid)
org_nrs_list = list(orgnrs_by_municipal_nr['orgnr'])
print(f"Number of org number for municipal {municipal_nr}: {len(org_nrs_list)}")
#print(orgnrs_by_municipal_nr.head())
polygons_by_municipal_nr, polygon_indices = get_polygons_by_municipal(filtered_polygons, municipal_nr)
print(f"Number of polygons for municipal {municipal_nr}: {len(polygons_by_municipal_nr)}")

transformed_polygons = convert_crs(polygons_by_municipal_nr)
print(len(transformed_polygons))
print(len(polygon_indices))
#write_polygons_to_shp(transformed_polygons, str(municipal_nr))

#polygons_plt = [Polygon(mapping(poly)['coordinates'][0][0]) for poly in transformed_polygons]
#plot_polygons(polygons_plt)
#print(orgnrs_by_municipal_nr.head())
for orgnr in org_nrs_list:
    row = orgnrs_by_municipal_nr.loc[orgnrs_by_municipal_nr['orgnr'] == orgnr].iloc[0]

    bounding_box = boundingBox(row['latitude'], row['longitude'], 1)
    bbox = box(bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3])

    write_polygons_to_shp([bbox], 'bbox' + orgnr)

    


#plot_polygons([bbox])


18
Number of org number for municipal 101: 165
Number of polygons for municipal 101: 0
0
0


In [106]:
orgnrs = filtered_farmer_centroid['orgnr'].tolist()

polygons = []
for nr in tqdm(orgnrs):
    lat, lng = filtered_farmer_centroid.loc[filtered_farmer_centroid['orgnr'] == nr].iloc[0]['latitude'], filtered_farmer_centroid.loc[filtered_farmer_centroid['orgnr'] == nr].iloc[0]['longitude']
    bounding_box = boundingBox(lat, lng, 1)
    bbox = box(bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3])
    polygons.append(bbox)





100%|██████████| 6738/6738 [00:06<00:00, 1062.43it/s]


In [107]:
write_polygons_to_shp(polygons, 'bounding_boxes')

In [126]:
print("Retrieving disposed properties data, satellite data and polygon data")
disp_eien = get_disp_eiendommer()
polygons = get_polygon_data()

sat_orgnr = np.array(get_combined_satellite_data())


Retrieving disposed properties data, satellite data and polygon data


100%|██████████| 3477/3477 [00:00<00:00, 34995.14it/s]


In [169]:
farm_orgnr = np.array(list(disp_eien['orgnr']))
intersection = np.intersect1d(sat_orgnr, farm_orgnr)

filtered_disp_eien = disp_eien[disp_eien['orgnr'].isin(intersection)]
filtered_satellite_data = intersection[:]

print(f"Farmer disposed properties shape filtering: {filtered_disp_eien.shape}")
print(f"Length of satellite data after filtering: {len(filtered_satellite_data)}")
print(f"Polygon data shape: {polygons.shape}")



municipal_nrs = []
idxs_to_remove = []
orgnrs_to_check = list(set(filtered_farmer_centroid['orgnr'].tolist()))
for index, row in tqdm(filtered_disp_eien.iterrows(), total=filtered_disp_eien.shape[0]):
    if row['orgnr'] in orgnrs_to_check:
        municipal_nrs.append(filtered_farmer_centroid.loc[filtered_farmer_centroid['orgnr'] == row['orgnr']]['kommunenr'].iloc[0])
    else:
        idxs_to_remove.append(index)
    
filtered_disp_eien = filtered_disp_eien.drop(idxs_to_remove)



Farmer disposed properties shape filtering: (17884, 3)
Length of satellite data after filtering: 6890
Polygon data shape: (629040, 3)


100%|██████████| 17884/17884 [00:08<00:00, 2065.81it/s]

(17884, 3)
(16857, 3)
16857


In [119]:
filtered_disp_eien.head()

,orgnr,year,geometry
0,969102404,2017,"MULTIPOLYGON (((199287.149 6584342.538, 199295..."
1,983375782,2017,"MULTIPOLYGON (((284208.764 6624834.683, 284208..."
3,983375782,2019,"MULTIPOLYGON (((284208.764 6624834.683, 284208..."
6,983375782,2018,"MULTIPOLYGON (((284208.764 6624834.683, 284208..."
11,971214074,2019,"MULTIPOLYGON (((283304.242 6602938.198, 283294..."


In [171]:
filtered_disp_eien.insert(1, "municipal_id", municipal_nrs)

ValueError: cannot insert municipal_id, already exists

In [120]:
filtered_polygons.head()

,Unnamed: 0,KOMID,geometry
0,0,3031,"MULTIPOLYGON (((271721.06120 6664057.75200, 27..."
1,1,3031,"MULTIPOLYGON (((271483.64850 6664134.17310, 27..."
2,2,3031,"MULTIPOLYGON (((271708.79830 6664057.24050, 27..."
3,3,3031,"MULTIPOLYGON (((271314.24970 6664101.64960, 27..."
4,4,3031,"MULTIPOLYGON (((271293.07800 6664065.84230, 27..."


In [186]:
intersections_df = []
disp_orgnrs = filtered_disp_eien['orgnr'].tolist()

for orgnr in tqdm(disp_orgnrs):
    poly_prop = filtered_disp_eien.loc[filtered_disp_eien['orgnr'] == orgnr].iloc[0]['geometry']
    municipal_nr = filtered_disp_eien.loc[filtered_disp_eien['orgnr'] == orgnr].iloc[0]['municipal_id']
    poly_fields = filtered_polygons.loc[filtered_polygons['KOMID'] == municipal_nr]['geometry'].tolist()

    
    for i, poly_field in enumerate(poly_fields):
        if poly_prop.intersects(poly_field):
            intersections_df.append([orgnr, i])

    


intersections_df = pd.DataFrame(intersections_df, columns=['orgnr', 'field_id'])
intersections_df.to_csv(os.path.join(data_location, 'farm-information/fields_per_farm2.csv'))
print(intersections_df.head())
print(intersections_df.shape)

  3%|▎         | 431/16857 [01:18<49:56,  5.48it/s]   


KeyboardInterrupt: 